In [1]:
# libraries import

import pandas as pd 
import requests
import json
from pandas import json_normalize
import time
from datetime import datetime
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from typing import Any, Dict, List
import boto3
import os



/Users/marieagrapart/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# city list given by the exercice

city_list = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
# request for lat & long for each city

position_cities = {}

for city in city_list:
    resp =  requests.get(f"https://nominatim.openstreetmap.org/?q={city}&format=json&countrycodes=1974&limit=1").json()
    lat = resp[0]['lat']
    long = resp[0]['lon']
    position_cities[city] = [lat,long]

position_cities

{'Mont Saint Michel': ['48.6359541', '-1.511459954959514'],
 'St Malo': ['48.649518', '-2.0260409'],
 'Bayeux': ['49.2764624', '-0.7024738'],
 'Le Havre': ['49.4938975', '0.1079732'],
 'Rouen': ['49.4404591', '1.0939658'],
 'Paris': ['48.8588897', '2.3200410217200766'],
 'Amiens': ['49.8941708', '2.2956951'],
 'Lille': ['50.6365654', '3.0635282'],
 'Strasbourg': ['48.584614', '7.7507127'],
 'Chateau du Haut Koenigsbourg': ['48.249489800000006', '7.34429620253195'],
 'Colmar': ['48.0777517', '7.3579641'],
 'Eguisheim': ['48.0447968', '7.3079618'],
 'Besancon': ['47.2380222', '6.0243622'],
 'Dijon': ['47.3215806', '5.0414701'],
 'Annecy': ['45.8992348', '6.1288847'],
 'Grenoble': ['45.1875602', '5.7357819'],
 'Lyon': ['45.7578137', '4.8320114'],
 'Gorges du Verdon': ['43.7496562', '6.3285616'],
 'Bormes les Mimosas': ['43.1506968', '6.3419285'],
 'Cassis': ['43.2140359', '5.5396318'],
 'Marseille': ['43.2961743', '5.3699525'],
 'Aix en Provence': ['43.5298424', '5.4474738'],
 'Avignon': 

In [4]:
# request for the weather for each city 

def get_weather_forecasts(latitude: float, longitude: float):
    resp =  requests.get(f"http://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid=c57023819bfd474995f3bf1b69e50c0d")
    resp.raise_for_status()
    return resp.json()['list']

# Keep usefull information, in our case I choose the precipiration & the temperature

chosen_weather = []

for city, (latitude, longitude) in position_cities.items():
    weather_forecasts = get_weather_forecasts(latitude, longitude)

    for forecast in weather_forecasts: 
        dt_object = datetime.utcfromtimestamp(forecast['dt'])
        main_weather = forecast['weather'][0]['main']
        prepcipitation = forecast['pop']
        temperature = forecast['main']['temp'] - 273.15
    
        chosen_weather.append({'city' : city,
                        'dt_object': dt_object,
                        'latitude': latitude,
                        'longitude': longitude,
                        'main_weather': main_weather, 
                        'prepcipitation' :prepcipitation,
                        'temperature' :temperature})

chosen_weather


[{'city': 'Mont Saint Michel',
  'dt_object': datetime.datetime(2023, 2, 15, 12, 0),
  'latitude': '48.6359541',
  'longitude': '-1.511459954959514',
  'main_weather': 'Clouds',
  'prepcipitation': 0,
  'temperature': 11.390000000000043},
 {'city': 'Mont Saint Michel',
  'dt_object': datetime.datetime(2023, 2, 15, 15, 0),
  'latitude': '48.6359541',
  'longitude': '-1.511459954959514',
  'main_weather': 'Clouds',
  'prepcipitation': 0,
  'temperature': 12.050000000000011},
 {'city': 'Mont Saint Michel',
  'dt_object': datetime.datetime(2023, 2, 15, 18, 0),
  'latitude': '48.6359541',
  'longitude': '-1.511459954959514',
  'main_weather': 'Clouds',
  'prepcipitation': 0,
  'temperature': 10.020000000000039},
 {'city': 'Mont Saint Michel',
  'dt_object': datetime.datetime(2023, 2, 15, 21, 0),
  'latitude': '48.6359541',
  'longitude': '-1.511459954959514',
  'main_weather': 'Clouds',
  'prepcipitation': 0,
  'temperature': 8.129999999999995},
 {'city': 'Mont Saint Michel',
  'dt_object':

In [5]:
# we tranform our data in dataframe

city_weather = pd.DataFrame.from_records(chosen_weather)
city_weather

,city,dt_object,latitude,longitude,main_weather,prepcipitation,temperature
0,Mont Saint Michel,2023-02-15 12:00:00,48.6359541,-1.511459954959514,Clouds,0.00,11.39
1,Mont Saint Michel,2023-02-15 15:00:00,48.6359541,-1.511459954959514,Clouds,0.00,12.05
2,Mont Saint Michel,2023-02-15 18:00:00,48.6359541,-1.511459954959514,Clouds,0.00,10.02
3,Mont Saint Michel,2023-02-15 21:00:00,48.6359541,-1.511459954959514,Clouds,0.00,8.13
4,Mont Saint Michel,2023-02-16 00:00:00,48.6359541,-1.511459954959514,Clouds,0.00,8.89
...,...,...,...,...,...,...,...
1395,La Rochelle,2023-02-19 21:00:00,46.1591126,-1.1520434,Clouds,0.02,9.96
1396,La Rochelle,2023-02-20 00:00:00,46.1591126,-1.1520434,Clouds,0.00,9.65
1397,La Rochelle,2023-02-20 03:00:00,46.1591126,-1.1520434,Clouds,0.00,9.30
1398,La Rochelle,2023-02-20 06:00:00,46.1591126,-1.1520434,Clouds,0.00,9.02


In [6]:
# tranform the infomation about city weather by days and not anymore by each 3h

city_weather['latitude'] = city_weather['latitude'].astype(float)
city_weather['longitude'] = city_weather['longitude'].astype(float)

city_weather_by_day = city_weather.groupby([city_weather['city'],city_weather['dt_object'].dt.date]).agg({ 
    'main_weather' : [pd.Series.mode], 
    'prepcipitation' : ['sum'], 
    'temperature' : ['mean'],
    'latitude' : ['mean'],
    'longitude' : ['mean']}).reset_index()

city_weather_by_day.columns = city_weather_by_day.columns.droplevel(1)

In [7]:
# send the data in our S3

session = boto3.Session(aws_access_key_id=os.getenv("AWS_ACCESS_LEY_ID"), 
                        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"))


s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket="booking-scapping")

csv = city_weather_by_day.to_csv()

put_object = bucket.put_object(Key="city_weather_by_day.csv", Body=csv)

In [8]:
# creation of a file with the top 5 to feed the booking scrapping

top_city = city_weather_by_day.groupby('city', sort=False).mean().reset_index().sort_values(['prepcipitation','temperature'], ascending = [True, False])

Top_city = top_city['city']
Top_5_city = Top_city.head(5)

Top_5_city.to_string('top_5_city_name.csv', index=False)